In [1]:
#@title Install PyDDM and download the script containing the models
!pip -q install git+https://github.com/mwshinn/PyDDM
import hashlib
import requests
import os

fname = "shinn2020.py"
url = "https://raw.githubusercontent.com/mwshinn/PyDDM/master/doc/downloads/shinn2021.py"

if not os.path.isfile(fname):
    r = requests.get(url)
    if r.status_code != requests.codes.ok:
      print("!!! Failed to download data !!!")
    else:
      with open(fname, "wb") as fid:
        fid.write(r.content)

In [ ]:
DIPTYPE = 1 # Set to 1, 2, or 3 depending on which model you would like to visualize

In [ ]:
#@title Visualize the models from Shinn et al (2021) - Transient neuronal suppression for exploitation of new sensory evidence
import ddm
import ddm.plot
from shinn2021 import DriftDip, NoiseDip, ICPoint, BoundDip, OverlayDipRatio

DIPTYPE = 1 # Change to 1, 2, or 3 depending on which model you want
snr = ddm.Fittable(minval=0.5, maxval=20, default=9.243318909157688)
leak = ddm.Fittable(minval=-10, maxval=30, default=9.46411355874963)
x0 = ddm.Fittable(minval=-.5, maxval=.5, default=0.1294632585920082)
leaktargramp = ddm.Fittable(minval=0, maxval=3, default=0)
noise = ddm.Fittable(minval=.2, maxval=2, default=1.1520906498077081)
t1 = ddm.Fittable(minval=0, maxval=1, default=0.34905555600815663)
t1slope = ddm.Fittable(minval=0, maxval=3, default=1.9643425020687162)

dipstart = ddm.Fittable(minval=-.4, maxval=0, default=-.2)
dipstop = ddm.Fittable(minval=0, maxval=.5, default=.05)
nondectime = ddm.Fittable(minval=0, maxval=.3, default=.1)
detect = ddm.Fittable(minval=2, maxval=50, default=10)
diptype = DIPTYPE
dipparam = ddm.Fittable(minval=0, maxval=50) if diptype == 2 else 0
pmixturecoef = ddm.Fittable(minval=0, maxval=.2, default=.03)
rate = ddm.Fittable(minval=.1, maxval=10, default=1)
m = ddm.Model(drift=                 DriftDip(snr=snr,
                                              noise=noise,
                                              t1=t1,
                                              t1slope=t1slope,
                                              leak=leak,
                                              maxcoh=70,
                                              leaktarget=x0,
                                              leaktargramp=leaktargramp,
                                              dipstart=dipstart,
                                              dipstop=dipstop,
                                              diptype=diptype,
                                              dipparam=dipparam,
                                              ),
              noise=                 NoiseDip(noise=noise,
                                              t1=t1,
                                              t1slope=t1slope,
                                              dipstart=dipstart,
                                              dipstop=dipstop,
                                              diptype=diptype,
                                              ),
              IC=                     ICPoint(x0=x0),
              bound=                 BoundDip(B=1,
                                              dipstart=dipstart,
                                              dipstop=dipstop,
                                              diptype=diptype
                                              ),
              overlay=ddm.OverlayChain(overlays=[
                                ddm.OverlayNonDecision(nondectime=nondectime),
                                        OverlayDipRatio(detect=detect,
                                                        diptype=diptype),
                              ddm.OverlayPoissonMixture(pmixturecoef=pmixturecoef,
                                                        rate=rate)
                                        ]),
              dx=0.002, dt=0.002, T_dur=3.0)
# END demo
ddm.plot.model_gui_jupyter(model=m, conditions={"coherence": [50, 53, 60, 70], "presample": [0, 400, 800], "highreward": [0, 1]})
